In [1]:
import logging
reload(logging)
log_fmt = '%(asctime)-9s %(levelname)-8s: %(message)s'
logging.basicConfig(format=log_fmt)

# Change to info once the notebook runs ok
logging.getLogger().setLevel(logging.DEBUG)

In [2]:
%pylab inline

import datetime
import devlib
import os
import json
import pandas as pd
import re
import subprocess
import trappy
from trappy.plotter.Utils import get_trace_event_data

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

# Support to access the remote target
#import devlib
#from env import TestEnv

from executor import Executor

Populating the interactive namespace from numpy and matplotlib


# Target Configuration

In [3]:
# Setup a target configuration
my_target_conf = {
    
    # Target platform and board
    "platform"    : 'linux',
    "board"       : 'juno',
    
    # Target board IP/MAC address
    "host"        : '10.0.0.57',
    
    # Login credentials
    "username"    : 'root',
    "password"    : 'insecure',

}

# Tests Configuration

In [4]:
my_tests_conf = {
    
    #/* Devlib modules to enable/disbale for all the experiments */
    "modules"         : [ "cpufreq", "cgroups" ],
    "exclude_modules" : [ "hwmon" ], # Disable hwmon - no DVFS

    #/* Binary tools required by the experiments */
    "tools"    : [ "rt-app" ],

    #/* FTrace configuration */
    "ftrace" : {
        "events" : [
            "sched_switch",
            "sched_contrib_scale_f",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_boostgroup_update",
            "sched_tune_filter",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_energy_diff",
            "cpu_frequency",
            "cpu_capacity",
            "cpu_idle",# Add cpuidle trace
            "sched_overutilized"
        ],
        "buffsize" : 10240,
    },

    #/* Set of platform configurations to test */
    "confs" : [
        {
            "tag" : "noboost",
            "flags" : "ftrace",
            "sched_features" : "NO_ENERGY_AWARE",        # Disable EAS!
#           "cpufreq" : { "governor" : "performance" },  #      "
            "cpufreq" : { "governor" : "ondemand" },     #
            "cgroups" : {
                "conf" : {
                    "schedtune" : {
                        "/"      : {"boost" :  0 },
                        "/stune" : {"boost" :  0 },
                    }
                },
                "default" : "/",
            }
        },

        {
            "tag" : "boost15",
            "flags" : "ftrace",
            "sched_features" : "ENERGY_AWARE",
            "cpufreq" : { "governor" : "sched" },
            "cgroups" : {
                "conf" : {
                    "schedtune" : {
                        "/"      : {"boost" :  0 },
                        "/stune" : {"boost" : 15 },
                    }
                },
                "default" : "/stune",
            }
        },
        {
            "tag" : "boost30",
            "flags" : "ftrace",
            "sched_features" : "ENERGY_AWARE",
            "cpufreq" : { "governor" : "sched" },
            "cgroups" : {
                "conf" : {
                    "schedtune" : {
                        "/"      : {"boost" :  0 },
                        "/stune" : {"boost" : 30 },
                    }
                },
                "default" : "/stune",
            }
        },
        {
            "tag" : "boost60",
            "flags" : "ftrace",
            "sched_features" : "ENERGY_AWARE",
            "cpufreq" : { "governor" : "sched" },
            "cgroups" : {
                "conf" : {
                    "schedtune" : {
                        "/"      : {"boost" :  0 },
                        "/stune" : {"boost" : 60 },
                    }
                },
                "default" : "/stune",
            }
        }

    ],

    #/* Set of workloads to run on each platform configuration */
    "wloads" : {
        "mixprof" : {
            "type": "rt-app",
            "conf" : {
                "class"  : "profile",
                "params"  : {
                    "r5_10-60" : {
                        "kind"   : "Ramp",
                        "params" : {
                            "period_ms" : 16,
                            "start_pct" :  5,
                            "end_pct"   : 60,
                            "delta_pct" :  5,
                            "time_s"    :  1,
                         }
                    }
                }
            },
            "loadref" : "LITTLE",
        }
    },

    #/* Number of iterations for each workload */
    "iterations" : 1,

}

# Tests execution

In [5]:
executor = Executor(my_target_conf, my_tests_conf)

02:52:51  INFO    :         Target - Loading custom (inline) test configuration
02:52:51  INFO    :         Target - Using base path: /home/ed/ar001/lisa
02:52:51  INFO    :         Target - Loading custom (inline) target configuration
02:52:51  INFO    :         Target - Loading custom (inline) test configuration
02:52:51  INFO    :         Target - Devlib modules to load: ['bl', 'cpufreq', 'cgroups']
02:52:51  INFO    :         Target - Connecting linux target:
02:52:51  INFO    :         Target -   username : root
02:52:51  INFO    :         Target -       host : 10.0.0.57
02:52:51  INFO    :         Target -   password : insecure
2016-06-14 14:52:51,620 DEBUG   : Logging in root@10.0.0.57
2016-06-14 14:52:53,641 DEBUG   : id
2016-06-14 14:52:54,077 DEBUG   : if [ -e '/root/devlib-target/bin' ]; then echo 1; else echo 0; fi
2016-06-14 14:52:54,490 DEBUG   : ls -1 /root/devlib-target/bin
2016-06-14 14:52:54,926 DEBUG   : cat /proc/cpuinfo
2016-06-14 14:52:55,351 DEBUG   : sudo -- sh 

In [6]:
executor.run()

02:55:04  INFO    : 
02:55:04  INFO    : ################################################################################
02:55:04  INFO    :       Executor - Experiments execution
02:55:04  INFO    : ################################################################################
02:55:04  INFO    : 
02:55:04  INFO    : ================================================================================
02:55:04  INFO    :   TargetConfig - configuring target for [noboost] experiments
02:55:04  INFO    :        CGroups - Initialize CGroups support...
02:55:04  INFO    :        CGroups - Setup [schedtune] controller...
2016-06-14 14:55:04,397 DEBUG   : sudo -- sh -c '[ -d /root/devlib-target/run_dir ] || mkdir /root/devlib-target/run_dir'
2016-06-14 14:55:05,304 DEBUG   : sudo -- sh -c 'grep schedtest /proc/mounts ||   mount -t tmpfs -o size=1024m schedtest /root/devlib-target/run_dir'
02:55:06  INFO    :  SchedFeatures - Set scheduler feature: NO_ENERGY_AWARE
2016-06-14 14:55:06,214 DEBUG 

In [7]:
!tree {executor.te.res_dir}

/home/ed/ar001/lisa/results/20160614_145346
├── rtapp:boost15:mixprof
│   ├── 1
│   │   ├── mixprof_00.json
│   │   ├── output.log
│   │   ├── rt-app-task_r5_10-60-0.log
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── rtapp:boost30:mixprof
│   ├── 1
│   │   ├── mixprof_00.json
│   │   ├── output.log
│   │   ├── rt-app-task_r5_10-60-0.log
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
├── rtapp:boost60:mixprof
│   ├── 1
│   │   ├── mixprof_00.json
│   │   ├── output.log
│   │   ├── rt-app-task_r5_10-60-0.log
│   │   └── trace.dat
│   ├── kernel.config
│   ├── kernel.version
│   └── platform.json
└── rtapp:noboost:mixprof
    ├── 1
    │   ├── mixprof_00.json
    │   ├── output.log
    │   ├── rt-app-task_r5_10-60-0.log
    │   └── trace.dat
    ├── kernel.config
    ├── kernel.version
    └── platform.json

8 directories, 28 files
